# PSF & Photometry Validation - SITCOM-426 

This notebook contains the execution of the PSF & Photometry Validation SITCOM-426 test described in https://jira.lsstcorp.org/browse/SITCOM-426, expected to run during the 2022-07A AuxTel Observing Run. 
                
This notebook is organized in  sections:

    i. Setup
    ii. Conditions assessment
    iii. CWFS
    iv. Find target object and slew.
    v. Confirm signal level
    vi. Perform the data acquisition sequence. 

## Set up

### Import libraries

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
from astropy.time import Time
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

import lsst_efd_client

from lsst.summit.extras import SpectralFocusAnalyzer

### Setting up logger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

### Getting unique index for script

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

### Instantiate CWFS Script

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [ ]:
atcs = script.atcs
latiss = script.latiss

### EFD 

In [3]:
efd_name = "summit_efd"
client = lsst_efd_client.EfdClient(efd_name)

### Set up script log level

In [ ]:
script.log.level = logging.DEBUG

### Write start info into EFD

In [ ]:
script.log.info(f'START- SITCOM-426 PSF&Photometry Validation -- at {Time.now()}')

## Assess that conditions meet criteria- sub-arcsec seeing and photometric
Photometric conditions clear definition?

## CWFS
A CWFS is recommended just before the test to ensure the system is in focus and the parabolic focus sweep is symmetrical.

### Slew to target

#### Declare target 

Query for a target with a magnitud limit of Vmag of 8 and around the same area of the sky as the target source.

In [ ]:
target = await script.atcs.find_target(az=180.2, el=75, mag_limit=8)
logger.info(f'Target for CWFS with mag_lim 8 is {target}')

#### Slew to the defined target

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

Uncomment the following line to take a snapshot to verify the target is positioned as expected 

In [ ]:
# exposure = await latiss.take_engtest(
#         exptime=5, n=1, filter='SDSSr', grating='empty_0', reason='Acquisition')
# logger.info(f'Acquisition exposure is {exposure}')

To offset the telescope and center the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

### Set up configuration

In [ ]:
configuration = yaml.safe_dump({"filter": 'SDSSr', 
                                "grating": 'empty_0',
                                "exposure_time": 20,})

The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [ ]:
await script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script

This sets the same group ID for all exposures taken in the script.

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

### Stop tracking

If required, then uncomment and use the following cell to stop the telescope from tracking, but you will lose your acquisition.

In [ ]:
# await script.atcs.stop_tracking()

---
## Find and slew to target object
"-35 < dec < -25 and hour angle 0:10:00 < HA < 0:20:00 (in other words an RA slightly less than LST so that it's just past the zenith and is setting) and mag 11< Vmag < 13. We want a target star that saturates in 30 sec. Patrick says                   saturation limit in 5 sec is   around 8th mag, so we should pick a target star that is (5/30~ 2 mag fainter).                 That should saturate in 50 sec. " 

### Declare target 

In [ ]:
azimuth = 180.2
elevation = 80
magnitude = 11
search_radius = 4

Query for target

In [ ]:
target = await script.atcs.find_target(az=azimuth, el=elevation, mag_limit=magnitude, radius = search_radius)
logger.info(f'Target object with mag_lim {magnitude} at {azimuth} Az and {elevation} EL deg is {target}')

### Slew to the defined target

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

To offset the telescope and move the source (if required) uncomment the following line. <br>
Offsets are in detector X/Y coordinates and in arcseconds. 

In [ ]:
# await script.atcs.offset_xy(x=20, y=20)

---
## Check signal level
Take a snapshot to verify the target saturates as expected, at 30 second exposure time.

In [ ]:
saturation_test = await latiss.take_engtest(
        exptime=30, n=1, filter='SDSSr', grating='empty_0', reason='Exposure_Time_Test')
logger.info(f'Saturation test exposure is {saturation_test}')

----
## Data Acquisition Sequence

### Focus Sweep

#### Parabolic Focus Data Acquisition

Acquire data in steps of `z_offset_step` in the range of `z_offset_start` to `z_offset_end` relative to the current focus position.

In [ ]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
logger.info(f'Original focus offset is \n {original_focus_offset}')

In [ ]:
script.log.info(f'START -- Focus Determination SDSSr \n'
                    f'Time UTC -- {astropy.time.Time(astropy.time.Time.now())}') 

z_offset_start = -0.1 # mm
z_offset_step = 0.025 # mm
z_offset_end = -z_offset_start

steps = np.arange(z_offset_start, z_offset_end + 0.01, z_offset_step)
parabolic_focus_data = []

await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_start)
    
await asyncio.sleep(2)
for step in steps:
        
    exposure = await latiss.take_engtest(
        exptime=20, n=1, filter='SDSSr', grating='empty_0', reason='Parabolic_focus_sequence-SITCOM-426')
    parabolic_focus_data.append(exposure[0])
    logger.info(f'Total z offset = {step:.3f} -- Image expId = {exposure[0]}')
    
    logger.info(f'Current focus offset is \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')
    
    await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_step)

# Move back to the original focus offset position
await atcs.rem.ataos.cmd_offset.set_start(z= -(z_offset_end + z_offset_step))
logger.info(f'Back to the original focus offset position \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')

# Log results into EFD for later analysis
script.log.info(f'END -- Focus Determination SDSSr \n'
                    f'Images expId -- {parabolic_focus_data} \n'
                    f'Time UTC -- {astropy.time.Time(astropy.time.Time.now())}') 

In [ ]:
current_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
logger.info(current_focus_offset)

In [ ]:
# Original and current focus offset's user comparison 

In [ ]:
logger.info(f'Current {current_focus_offset.userApplied} focus vs. original {original_focus_offset.userApplied} focus offsets')

#### Parabolic Data Analysis

In [2]:
%matplotlib inline

dayObs format is yyyymmdd. <br>
SeqNums is a list of integers, containing the first and last image of the focus sweep sequence. 

In [ ]:
dayObs = 20220715
seqNums = [s for s in range(286, 294+1)]
print(f'Observation day is {dayObs} \n'
      f'Sequence numbers are {seqNums}')

In case one or more of the images from the sequence are invalid, you can drop them from the seqNums array. Uncomment the cell below after you have replaced the images_to_discard values.   

In [ ]:
# images_to_discard = [286,288]
# for k in range(len(images_to_discard)):
#     try:
#         seqNums.remove(images_to_discard[k])
#     except:
#         print(f'{images_to_discard[k]} image not in original seqNums list')
        
print(f'\nObservation day is {dayObs} \n'
      f'New sequence numbers are {seqNums}')

In [ ]:
focusAnalyzer = NonSpectralFocusAnalyzer()

focusAnalyzer.getFocusData(dayObs, seqNums, doDisplay=True)
focusAnalyzer.fitDataAndPlot()

#### Set system to best focus, if required

In [ ]:
z_best_focus = 

In [ ]:
await atcs.rem.ataos.cmd_offset.set_start(z=z_best_focus)

### Image Sequence

In [ ]:
# 2 sec exposures
await latiss.take_engtest(
        exptime=2, n=25, filter='SDSSr', grating='empty_0', reason='SITCOM-426')

In [ ]:
# 10 sec exposures
await latiss.take_engtest(
        exptime=10, n=25, filter='SDSSr', grating='empty_0', reason='SITCOM-426')

In [ ]:
# 25 sec exposures
await latiss.take_engtest(
        exptime=25, n=10, filter='SDSSr', grating='empty_0', reason='SITCOM-426')

In [ ]:
# 50 sec exposures
await latiss.take_engtest(
        exptime=50, n=5, filter='SDSSr', grating='empty_0', reason='SITCOM-426')

In [ ]:
# 100 sec exposures
await latiss.take_engtest(
        exptime=100, n=3, filter='SDSSr', grating='empty_0', reason='SITCOM-426')

In [ ]:
# 2 sec exposures
await latiss.take_engtest(
        exptime=2, n=25, filter='SDSSr', grating='empty_0', reason='SITCOM-426')

In [ ]:
### Ending Test

In [ ]:
script.log.info(f'END- SITCOM-426 PSF&Photometry Validation -- at {Time.now()}')